In [ ]:
# Import libraries
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

# Set up matplotlib for inline plotting
%matplotlib inline

# Define the paths to the training and test data folders
training_data_folder_path = 'dataset/training-data'
test_data_folder_path = 'dataset/test-data'

# Load a random image from the training data
random_image = cv2.imread('dataset/training-data/0/Aya4.jpg')

# Display the image
fig = plt.figure()
ax1 = fig.add_axes((0.1, 0.2, 0.8, 0.7))
ax1.set_title('Image from category 0')  # Change category name accordingly
plt.imshow(cv2.cvtColor(random_image, cv2.COLOR_BGR2RGB))
plt.show()

# Load Haar cascade for face detection
haarcascade_frontalface = 'D:\\GradProj\\haarcascade_frontalface.xml'

# Face detection function
def detect_face(input_img):
    image = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(haarcascade_frontalface)
    
    # Check if the Haar cascade is loaded properly
    if face_cascade.empty():
        print("Error: Haar cascade not loaded. Check the file path.")
        return -1, -1

    faces = face_cascade.detectMultiScale(image, scaleFactor=1.2, minNeighbors=5)
    
    if len(faces) == 0:
        return -1, -1

    (x, y, w, h) = faces[0]
    return input_img[y:y+h, x:x+w], faces[0]  # Return the original color image

# Prepare training dataset
def prepare_training_data(training_data_folder_path):
    detected_faces = []
    face_labels = []
    training_image_dirs = os.listdir(training_data_folder_path)

    for dir_name in training_image_dirs:
        try:
            label = int(dir_name)  # Ensure the directory name is numeric
        except ValueError:
            print(f"Skipping directory '{dir_name}', it is not numeric.")
            continue

        training_image_path = os.path.join(training_data_folder_path, dir_name)
        training_images_names = os.listdir(training_image_path)

        for image_name in training_images_names:
            image_path = os.path.join(training_image_path, image_name)
            image = cv2.imread(image_path)

            if image is None:
                print(f"Error loading image: {image_path}")
                continue

            face, rect = detect_face(image)
            if face != -1:
                resized_face = cv2.resize(face, (121, 121), interpolation=cv2.INTER_AREA)
                detected_faces.append(resized_face)
                face_labels.append(label)

    return detected_faces, face_labels

# Prepare training data
detected_faces, face_labels = prepare_training_data(training_data_folder_path)
print("Total faces: ", len(detected_faces))
print("Total labels: ", len(face_labels))

# Initialize a face recognizer
fisherfaces_recognizer = cv2.face.FisherFaceRecognizer_create()

# Train the face recognizer model
fisherfaces_recognizer.train(detected_faces, np.array(face_labels))

def draw_rectangle(test_image, rect):
    (x, y, w, h) = rect
    cv2.rectangle(test_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

def draw_text(test_image, label_text, x, y):
    cv2.putText(test_image, label_text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

# Predict output on test data
def predict(test_image):
    detected_face, rect = detect_face(test_image)
    
    if detected_face == -1:
        print("No face detected.")
        return test_image, "Unknown"
    
    resized_face = cv2.resize(detected_face, (121, 121), interpolation=cv2.INTER_AREA)
    label = fisherfaces_recognizer.predict(resized_face)
    
    label_text = tags[label[0]]
    draw_rectangle(test_image, rect)
    draw_text(test_image, label_text, rect[0], rect[1] - 5)
    
    return test_image, label_text

# Define tags for the recognized classes
tags = ['0', '1']

# Load a test image
test_image = cv2.imread("dataset/test-data/0/Aya5.jpg")
predicted_image, label = predict(test_image)

# Display the predicted image with labels
fig = plt.figure()
ax1 = fig.add_axes((0.1, 0.2, 0.8, 0.7))
ax1.set_title('actual class: ' + tags[2] + ' | ' + 'predicted class: ' + label)
plt.axis("off")
plt.imshow(cv2.cvtColor(predicted_image, cv2.COLOR_BGR2RGB))
plt.show()
